The following additional libraries are needed to run this
notebook. Note that running on Colab is experimental, please report a Github
issue if you have any problem.

In [1]:
%%capture

import sys
sys.path.append('..')
import mock_d2l_jax as d2l

# Batch Normalization
:label:`sec_batch_norm`

Training deep neural networks is difficult.
Getting them to converge in a reasonable amount of time can be tricky.
In this section, we describe *batch normalization*, a popular and effective technique
that consistently accelerates the convergence of deep networks :cite:`Ioffe.Szegedy.2015`.
Together with residual blocks---covered later in :numref:`sec_resnet`---batch normalization
has made it possible for practitioners to routinely train networks with over 100 layers.
A secondary (serendipitous) benefit of batch normalization is its inherent regularization. 

## Training Deep Networks

When working with data, we often preprocess before training. 
Choices regarding data preprocessing often make an enormous difference in the final results.
Recall our application of MLPs to predicting house prices (:numref:`sec_kaggle_house`).
Our first step when working with real data
was to standardize our input features to have 
zero mean $\mathbf{\mu} = 0$ and unit variance $\mathbf{\Sigma} = \mathbf{1}$ across multiple observations :cite:`friedman1987exploratory`.
At a mimimum, one frequently rescales it such that the diagonal is unity, i.e., $\Sigma_{ii} = 1$. 
Yet another strategy is to rescale vectors to unit length, possibly zero mean *per observation*. 
This can work well, e.g., for spatial sensor data. These preprocessing techniques and many more are 
beneficial to keep the estimation problem well controlled. See e.g., the articles in :cite:`guyon2008feature` for a review of feature selection and extraction techniques.

Intuitively, this standardization plays nicely with our optimizers
since it puts the parameters *a priori* at a similar scale.
As such, it is only natural to ask whether a corresponding normalization step *inside* a deep network
might not be beneficial. While this isn't quite the reasoning that led to the invention of batch normalization :cite:`Ioffe.Szegedy.2015`, it is a useful way of understanding it and its cousin, layer normalization :cite:`Ba.Kiros.Hinton.2016` within a unified framework. 

Second, for a typical MLP or CNN, as we train,
the variables (e.g., affine transformation outputs in MLP)
in intermediate layers 
may take values with widely varying magnitudes:
both along the layers from the input to the output, across units in the same layer,
and over time due to our updates to the model parameters.
The inventors of batch normalization postulated informally
that this drift in the distribution of such variables could hamper the convergence of the network.
Intuitively, we might conjecture that if one
layer has variable activations that are 100 times that of another layer,
this might necessitate compensatory adjustments in the learning rates. Adaptive solvers 
such as AdaGrad :cite:`Duchi.Hazan.Singer.2011`, Adam :cite:`Kingma.Ba.2014`, and Yogi :cite:`Zaheer.Reddi.Sachan.ea.2018` aim to address this from the viewpoint of optimization. 
The alternative is to prevent the problem from occurring, simply by adaptive normalization.
   
Third, deeper networks are complex and tend to be more easily capable of overfitting.
This means that regularization becomes more critical. A common technique for regularization is noise 
injection. This has been known for a long time, e.g., with regard to noise injection for the 
inputs :cite:`Bishop.1995`. It also forms the basis of dropout :numref:`sec_dropout`. As it turns out, quite serendipitously, batch normalization conveys all three benefits: preprocessing, numerical stability, and regularization. 

Batch normalization is applied to individual layers, or optionally, to all of them:
In each training iteration,
we first normalize the inputs (of batch normalization)
by subtracting their mean and
dividing by their standard deviation,
where both are estimated based on the statistics of the current minibatch.
Next, we apply a scale coefficient and an offset to recover the lost degrees 
of freedom. It is precisely due to this *normalization* based on *batch* statistics
that *batch normalization* derives its name.

Note that if we tried to apply batch normalization with minibatches of size 1,
we would not be able to learn anything.
That is because after subtracting the means,
each hidden unit would take value 0.
As you might guess, since we are devoting a whole section to batch normalization,
with large enough minibatches, the approach proves effective and stable.
One takeaway here is that when applying batch normalization,
the choice of batch size is
even more significant than without batch normalization, or at least, 
suitable calibration is needed as we might adjust it.

Formally, denoting by $\mathbf{x} \in \mathcal{B}$ an input to batch normalization ($\mathrm{BN}$)
that is from a minibatch $\mathcal{B}$,
batch normalization transforms $\mathbf{x}$
according to the following expression:

$$\mathrm{BN}(\mathbf{x}) = \boldsymbol{\gamma} \odot \frac{\mathbf{x} - \hat{\boldsymbol{\mu}}_\mathcal{B}}{\hat{\boldsymbol{\sigma}}_\mathcal{B}} + \boldsymbol{\beta}.$$
:eqlabel:`eq_batchnorm`

In :eqref:`eq_batchnorm`,
$\hat{\boldsymbol{\mu}}_\mathcal{B}$ is the  sample mean
and $\hat{\boldsymbol{\sigma}}_\mathcal{B}$ is the sample standard deviation of the minibatch $\mathcal{B}$.
After applying standardization,
the resulting minibatch
has zero mean and unit variance. 
The choice of unit variance
(vs. some other magic number) is an arbitrary choice. We recover this degree of freedom
by including an elementwise
*scale parameter* $\boldsymbol{\gamma}$ and *shift parameter* $\boldsymbol{\beta}$
that have the same shape as $\mathbf{x}$. Both are parameters that 
need to be learned as part of model training.

Consequently, the variable magnitudes
for intermediate layers cannot diverge during training
because batch normalization actively centers and rescales them back
to a given mean and size (via $\hat{\boldsymbol{\mu}}_\mathcal{B}$ and ${\hat{\boldsymbol{\sigma}}_\mathcal{B}}$).
Practical experience confirms that, as alluded when discussing feature rescaling, batch normalization seems to allow for more aggressive learning rates.
Formally, 
we calculate $\hat{\boldsymbol{\mu}}_\mathcal{B}$ and ${\hat{\boldsymbol{\sigma}}_\mathcal{B}}$ in :eqref:`eq_batchnorm` as follows:

$$\hat{\boldsymbol{\mu}}_\mathcal{B} = \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} \mathbf{x}
\text{ and }
\hat{\boldsymbol{\sigma}}_\mathcal{B}^2 = \frac{1}{|\mathcal{B}|} \sum_{\mathbf{x} \in \mathcal{B}} (\mathbf{x} - \hat{\boldsymbol{\mu}}_{\mathcal{B}})^2 + \epsilon.$$

Note that we add a small constant $\epsilon > 0$
to the variance estimate
to ensure that we never attempt division by zero,
even in cases where the empirical variance estimate might be very small or even vanish, leading to a division by zero.
The estimates $\hat{\boldsymbol{\mu}}_\mathcal{B}$ and ${\hat{\boldsymbol{\sigma}}_\mathcal{B}}$ counteract the scaling issue
by using noisy estimates of mean and variance.
You might think that this noisiness should be a problem.
As it turns out, this is actually beneficial.

This turns out to be a recurring theme in deep learning.
For reasons that are not yet well-characterized theoretically,
various sources of noise in optimization
often lead to faster training and less overfitting:
this variation appears to act as a form of regularization.
:cite:`Teye.Azizpour.Smith.2018` and :cite:`Luo.Wang.Shao.ea.2018`
relate the properties of batch normalization to Bayesian priors and penalties respectively.
In particular, this sheds some light on the puzzle
of why batch normalization works best for moderate minibatches sizes in the $50 \sim 100$ range. 
This particular size of minibatch seems to inject just the "right amount" of noise per layer: a 
larger minibatch regularizes less due to the more stable estimates, whereas tiny minibatches 
destroy useful signal due to high variance. Exploring this direction further, considering alternative types 
of preprocessing and filtering may yet lead to other effective types of regularization. 

Fixing a trained model, you might think
that we would prefer using the entire dataset
to estimate the mean and variance.
Once training is complete, why would we want
the same image to be classified differently,
depending on the batch in which it happens to reside?
During training, such exact calculation is infeasible
because the intermediate variables
for all data examples
change every time we update our model.
However, once the model is trained,
we can calculate the means and variances
of each layer's variables based on the entire dataset.
Indeed this is standard practice for
models employing batch normalization
and thus batch normalization layers function differently
in *training mode* (normalizing by minibatch statistics)
and in *prediction mode* (normalizing by dataset statistics). 
In this form they closely resemble the behavior of dropout regularization of :numref:`sec_dropout`,
where noise is only injected during training. 


## Batch Normalization Layers

We are now ready to take a look at how batch normalization works in practice.
Batch normalization implementations for fully connected layers
and convolutional layers are slightly different.
One key difference between batch normalization and other layers
is that because batch normalization operates on a full minibatch at a time,
we cannot just ignore the batch dimension
as we did before when introducing other layers.

### Fully Connected Layers

When applying batch normalization to fully connected layers,
the original paper inserted batch normalization after the affine transformation
and before the nonlinear activation function. Later applications experimented with 
inserting batch normalization right after activation functions :cite:`Ioffe.Szegedy.2015`.
Denoting the input to the fully connected layer by $\mathbf{x}$,
the affine transformation
by $\mathbf{W}\mathbf{x} + \mathbf{b}$ (with the weight parameter $\mathbf{W}$ and the bias parameter $\mathbf{b}$),
and the activation function by $\phi$,
we can express the computation of a batch-normalization-enabled,
fully connected layer output $\mathbf{h}$ as follows:

$$\mathbf{h} = \phi(\mathrm{BN}(\mathbf{W}\mathbf{x} + \mathbf{b}) ).$$

Recall that mean and variance are computed
on the *same* minibatch 
on which the transformation is applied.

### Convolutional Layers

Similarly, with convolutional layers,
we can apply batch normalization after the convolution
and before the nonlinear activation function. The key difference from batch normalization 
in fully connected layers is that we apply the operation on a per-channel basis 
*across all locations*. This is compatible with our assumption of translation 
invariance that led to convolutions: we assumed that the specific location of a pattern 
within an image was not critical for the purpose of understanding.

Assume that our minibatches contain $m$ examples
and that for each channel,
the output of the convolution has height $p$ and width $q$.
For convolutional layers, we carry out each batch normalization
over the $m \cdot p \cdot q$ elements per output channel simultaneously.
Thus, we collect the values over all spatial locations
when computing the mean and variance
and consequently 
apply the same mean and variance
within a given channel
to normalize the value at each spatial location.
Each channel has its own scale and shift parameters,
both of which are scalars.

Note that in the context of convolutions the batch normalization is well-defined even for 
minibatches of size 1: after all, we have all the locations across an image to average. Consequently, 
mean and variance are well defined, even if it's just within a single observation. This consideration 
led :cite:`Ba.Kiros.Hinton.2016` to introduce the notion of the *layer norm*. It works just like 
a batch norm, just that it is applied one image at a time. There are cases where layer normalization improves the 
accuracy of a model. We skip further details and recommend the interested reader to consult the 
original paper. 

### Batch Normalization During Prediction

As we mentioned earlier, batch normalization typically behaves differently
in training mode and prediction mode.
First, the noise in the sample mean and the sample variance
arising from estimating each on minibatches
are no longer desirable once we have trained the model.
Second, we might not have the luxury
of computing per-batch normalization statistics.
For example,
we might need to apply our model to make one prediction at a time.

Typically, after training, we use the entire dataset
to compute stable estimates of the variable statistics
and then fix them at prediction time.
Consequently, batch normalization behaves differently during training and at test time.
Recall that dropout also exhibits this characteristic.

## (**Implementation from Scratch**)

To see how batch normalization works in practice, we implement one from scratch below.


In [2]:
import jax
from jax import numpy as jnp, random, grad, vmap, jit
from flax import linen as nn
import optax
# from d2l import jax as d2l


def batch_norm(X, gamma, beta, moving_mean, moving_var, eps):
    # Compute reciprocal of square root of the moving variance elementwise
    inv = jax.lax.rsqrt(moving_var + eps).astype(X.dtype)
    # Scale and shift
    inv *= gamma
    Y = X * inv + (beta - moving_mean * inv)
    return Y

We can now [**create a proper `BatchNorm` layer.**]
Our layer will maintain proper parameters
for scale `gamma` and shift `beta`,
both of which will be updated in the course of training.
Additionally, our layer will maintain
moving averages of the means and variances
for subsequent use during model prediction.

Putting aside the algorithmic details,
note the design pattern underlying our implementation of the layer.
Typically, we define the mathematics in a separate function, say `batch_norm`.
We then integrate this functionality into a custom layer,
whose code mostly addresses bookkeeping matters,
such as moving data to the right device context,
allocating and initializing any required variables,
keeping track of moving averages (here for mean and variance), and so on.
This pattern enables a clean separation of mathematics from boilerplate code.
Also note that for the sake of convenience
we did not worry about automatically inferring the input shape here,
thus we need to specify the number of features throughout.
By now all modern deep learning frameworks offer automatic detection of size and shape in the 
high-level batch normalization APIs (in practice we will use this instead).


In [ ]:
class BatchNorm(nn.Module):
    input_shape: list

    def setup(self):
        weight_shape = [self.input_shape[-1], ]
        # The scale parameter and the shift parameter (model parameters) are
        # initialized to 1 and 0, respectively
        self.gamma = self.param('gamma', nn.initializers.ones, weight_shape)
        self.beta = self.param('beta', nn.initializers.zeros, weight_shape)
        # The variables that are not model parameters are initialized to 0
        self.moving_mean = self.variable('batch_stats', 'moving_mean', 
                                         nn.initializers.zeros, weight_shape)
        self.moving_variance = self.variable('batch_stats', 'moving_variance',
                                             nn.initializers.ones, weight_shape)

    def assign_moving_average(self, variable, value):
        momentum = 0.9
        delta = variable * momentum + value * (1 - momentum)
        return variable.assign(delta)

    def __call__(self, inputs, training):
        if training:
            axes = list(range(len(inputs.shape) - 1))
            batch_mean = jnp.mean(inputs, axes, keepdims=True)
            batch_variance = jnp.mean((inputs - jnp.stop_gradient(batch_mean)) ** 2, axes, keepdims=True)
            batch_mean = jnp.squeeze(batch_mean, axes)
            batch_variance = jnp.squeeze(batch_variance, axes)
            mean_update = self.assign_moving_average(
                self.moving_mean, batch_mean)
            variance_update = self.assign_moving_average(
                self.moving_variance, batch_variance)
            # self.add_update(mean_update) ???
            # self.add_update(variance_update)
            mean, variance = batch_mean, batch_variance
        else:
            mean, variance = self.moving_mean, self.moving_variance
        output = batch_norm(inputs, moving_mean=mean, moving_var=variance,
            beta=self.beta, gamma=self.gamma, eps=1e-5)
        return output

Note that we used the variable `momentum` to govern the aggregation over past mean and variance estimates. This is somewhat of a misnomer as it has nothing whatsoever to do with the *momentum* term of optimization in :numref:`sec_momentum`. Nonetheless, it is the commonly adopted name for this term and in deference to API naming convention we use the same variable name in our code, too.

## [**Applying Batch Normalization in LeNet**]

To see how to apply `BatchNorm` in context,
below we apply it to a traditional LeNet model (:numref:`sec_lenet`).
Recall that batch normalization is applied
after the convolutional layers or fully connected layers
but before the corresponding activation functions.


In [ ]:
class BNLeNetScratch(d2l.Classifier):
    lr: float = 0.1
    num_classes: int = 10

    def setup(self):
        avg_pool = lambda x: nn.avg_pool(x, kernel_size=(2, 2), strides=(2, 2))
        self.net = nn.Sequential([
            nn.Conv(6, kernel_size=(5, 5)), BatchNorm(6, num_dims=4),
            nn.sigmoid, avg_pool,
            nn.Conv(16, kernel_size=(5, 5)), BatchNorm(16, num_dims=4),
            nn.sigmoid, avg_pool,
            d2l.flatten, nn.Dense(120),
            BatchNorm(120, num_dims=2), nn.sigmoid, nn.Dense(84),
            BatchNorm(84, num_dims=2), nn.sigmoid,
            nn.Dense(self.num_classes)])

As before, we will [**train our network on the Fashion-MNIST dataset**].
This code is virtually identical to that when we first trained LeNet (:numref:`sec_lenet`).


In [ ]:
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128)
model = BNLeNetScratch(lr=0.1)
# model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
trainer.fit(model, data) # TODO: make from scratch work

Let's [**have a look at the scale parameter `gamma`
and the shift parameter `beta`**] learned
from the first batch normalization layer.


In [ ]:
model.net[1].gamma.reshape((-1,)), model.net[1].beta.reshape((-1,))

## [**Concise Implementation**]

Compared with the `BatchNorm` class,
which we just defined ourselves,
we can use the `BatchNorm` class defined in high-level APIs from the deep learning framework directly.
The code looks virtually identical
to our implementation above, except that we no longer need to provide additional arguments for it to get the dimensions right.


In [20]:
class BNLeNet(d2l.Classifier):
    lr: float = 0.1
    num_classes: int = 10
    training: bool = True

    def setup(self):
        avg_pool = lambda x: nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
        use_running_average = not self.training
        self.net = nn.Sequential([
            nn.Conv(6, kernel_size=(5, 5)), nn.BatchNorm(use_running_average),
            nn.sigmoid, avg_pool,
            nn.Conv(16, kernel_size=(5, 5)), nn.BatchNorm(use_running_average),
            nn.sigmoid, avg_pool,
            d2l.flatten, nn.Dense(120), nn.BatchNorm(use_running_average),
            nn.sigmoid, nn.Dense(84), nn.BatchNorm(use_running_average),
            nn.sigmoid, nn.Dense(self.num_classes)])


Below, we [**use the same hyperparameters to train our model.**]
Note that as usual, the high-level API variant runs much faster
because its code has been compiled to C++ or CUDA
while our custom implementation must be interpreted by Python.


In [ ]:
trainer = d2l.Trainer(max_epochs=10) # num_gpus=1
data = d2l.FashionMNIST(batch_size=128)
model = BNLeNet(lr=0.1)
# model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
trainer.fit(model, data, mutable=['batch_stats'])

## Discussion

Intuitively, batch normalization is thought
to make the optimization landscape smoother. 
However, we must be careful to distinguish between
speculative intuitions and true explanations
for the phenomena that we observe when training deep models.
Recall that we do not even know why simpler
deep neural networks (MLPs and conventional CNNs)
generalize well in the first place.
Even with dropout and weight decay,
they remain so flexible that their ability to generalize to unseen data
likely needs significantly more refined learning-theoretic generalization guarantees.

In the original paper proposing batch normalization,
the authors, in addition to introducing a powerful and useful tool,
offered an explanation for why it works:
by reducing *internal covariate shift*.
Presumably by *internal covariate shift* the authors
meant something like the intuition expressed above---the
notion that the distribution of variable values changes
over the course of training.
However, there were two problems with this explanation:
i) This drift is very different from *covariate shift*,
rendering the name a misnomer.
ii) The explanation offers an under-specified intuition
but leaves the question of *why precisely this technique works*
an open question wanting for a rigorous explanation.
Throughout this book, we aim to convey the intuitions that practitioners
use to guide their development of deep neural networks.
However, we believe that it is important
to separate these guiding intuitions
from established scientific fact.
Eventually, when you master this material
and start writing your own research papers
you will want to be clear to delineate
between technical claims and hunches.

Following the success of batch normalization,
its explanation in terms of *internal covariate shift*
has repeatedly surfaced in debates in the technical literature
and broader discourse about how to present machine learning research.
In a memorable speech given while accepting a Test of Time Award
at the 2017 NeurIPS conference,
Ali Rahimi used *internal covariate shift*
as a focal point in an argument likening
the modern practice of deep learning to alchemy.
Subsequently, the example was revisited in detail
in a position paper outlining
troubling trends in machine learning :cite:`Lipton.Steinhardt.2018`.
Other authors
have proposed alternative explanations for the success of batch normalization,
some claiming that batch normalization's success comes despite exhibiting behavior
that is in some ways opposite to those claimed in the original paper :cite:`Santurkar.Tsipras.Ilyas.ea.2018`.

We note that the *internal covariate shift*
is no more worthy of criticism than any of
thousands of similarly vague claims
made every year in the technical machine learning literature.
Likely, its resonance as a focal point of these debates
owes to its broad recognizability to the target audience.
Batch normalization has proven an indispensable method,
applied in nearly all deployed image classifiers,
earning the paper that introduced the technique
tens of thousands of citations. We conjecture, though, that the guiding principles 
of regularization through noise injection, acceleration through rescaling and lastly preprocessing
may well lead to further inventions of layers and techniques in the future. 

On a more practical note, there are a number of aspects worth remembering about batch normalization: 
* During model training, batch normalization continuously adjusts the intermediate output of 
  the network by utilizing the mean and standard deviation of the minibatch, so that the 
  values of the intermediate output in each layer throughout the neural network are more stable.
* Batch normalization for fully connected layers and convolutional layers are slightly different. In fact, 
  for convolutional layers, layer normalization can sometimes be used as an alternative. 
* Like a dropout layer, batch normalization layers have different behaviors 
  in training mode and prediction mode.
* Batch normalization is useful for regularization and improving convergence in optimization. On the other hand, 
  the original motivation of reducing internal covariate shift seems not to be a valid explanation.

## Exercises

1. Can we remove the bias parameter from the fully connected layer or the convolutional layer before the batch normalization? Why?
1. Compare the learning rates for LeNet with and without batch normalization.
    1. Plot the increase in validation accuracy.
    1. How large can you make the learning rate before the optimization fails in both cases?
1. Do we need batch normalization in every layer? Experiment with it?
1. Implement a "lite" version of batch normalization that only removes the mean, or alternatively one that 
   only removes the variance. How does it behave?
1. Fix the parameters `beta` and `gamma`, and observe and analyze the results.
1. Can you replace dropout by batch normalization? How does the behavior change?
1. Review the online documentation for `BatchNorm` from the high-level APIs to see 
   some other use cases for it. 
1. Research ideas: think of other normalization transforms that you can apply:
    1. Can you apply the probability integral transform? 
    1. Can you use a full rank covariance estimate? Why not?
    1. Does a sparsification compression act as a regularizer?


[Discussions](https://discuss.d2l.ai/t/84)
